In [18]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import glob
from metavision_core.event_io import EventsIterator

In [19]:
# ---------------------------
# Load images for homography mapping
# ---------------------------
basler = cv2.imread("Calibration/basler_image_10.png")
prophesee = cv2.imread("Calibration/prophesee_image_10.png")

# Flip Prophesee (due to prism)
prophesee_flipped = cv2.flip(prophesee, 1)

In [20]:
# ---------------------------
# Manual point picking
# ---------------------------

def pick_points(image, window_name="Image", display_scale=0.5):
    original = image.copy()
    display_img = cv2.resize(image, (0, 0), fx=display_scale, fy=display_scale)
    clone = display_img.copy()
    points = []

    def click_event(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            scaled_x, scaled_y = int(x / display_scale), int(y / display_scale)
            points.append([scaled_x, scaled_y])
            print(f"Clicked: ({scaled_x}, {scaled_y})")
            cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow(window_name, clone)

    cv2.imshow(window_name, clone)
    cv2.setMouseCallback(window_name, click_event)

    print(f"Click points on {window_name}. Press any key when done...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return np.array(points, dtype=np.float32)

# Pick points on Basler
basler_pts = pick_points(basler, "Basler Image", display_scale=0.5)

# Pick matching points on Prophesee flipped
prophesee_pts = pick_points(prophesee_flipped, "Prophesee Flipped", display_scale=1.0)

print("\nBasler points:\n", basler_pts)
print("\nProphesee points:\n", prophesee_pts)


✅ Click points on Basler Image. Press any key when done...
Clicked: (430, 416)
Clicked: (1182, 444)
Clicked: (418, 788)
Clicked: (1154, 804)
✅ Click points on Prophesee Flipped. Press any key when done...
Clicked: (486, 254)
Clicked: (1152, 262)
Clicked: (481, 579)
Clicked: (1141, 582)

✅ Basler points:
 [[ 430.  416.]
 [1182.  444.]
 [ 418.  788.]
 [1154.  804.]]

✅ Prophesee points:
 [[ 486.  254.]
 [1152.  262.]
 [ 481.  579.]
 [1141.  582.]]


In [21]:
# ================================
# Compute homography
# ================================

H, status = cv2.findHomography(basler_pts, prophesee_pts, cv2.RANSAC)
print("\nHomography matrix:\n", H)

prophesee_height, prophesee_width = prophesee_flipped.shape[:2]

# Warp Basler to Prophesee size:
basler_warped = cv2.warpPerspective(basler, H, (prophesee_width, prophesee_height))



✅ Homography matrix:
 [[ 8.40784391e-01 -2.81774730e-03  1.14305605e+02]
 [-2.61258370e-02  8.32209434e-01 -8.68860834e+01]
 [-1.97802318e-05 -3.55908538e-05  1.00000000e+00]]


In [22]:
# Visual overlay check
overlay = cv2.addWeighted(basler_warped, 0.5, prophesee_flipped, 0.5, 0)

cv2.imshow("Basler Warped to Prophesee", basler_warped)
cv2.imshow("Prophesee Flipped", prophesee_flipped)
cv2.imshow("Overlay", overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save
cv2.imwrite("basler_warped_10.png", basler_warped)
cv2.imwrite("overlay_10.png", overlay)
cv2.imwrite("prophesee_flipped_10.png", prophesee_flipped)

True

In [23]:
# =================
# Cropping mapped images for display only
# =================
# Find mask of non-black pixels
gray = cv2.cvtColor(basler_warped, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

# Find bounding rectangle of valid data
x, y, w, h = cv2.boundingRect(mask)

# Crop to valid region
basler_warped_cropped = basler_warped[y:y+h, x:x+w]
prophesee_cropped = prophesee_flipped[y:y+h, x:x+w]


In [24]:
# =============
# Display and save cropped/warped homography mapping images
# =============

# Redo overlay
overlay_cropped = cv2.addWeighted(basler_warped_cropped, 0.5, prophesee_cropped, 0.5, 0)

# Show cropped overlay
cv2.imshow("Cropped Basler Warped", basler_warped_cropped)
cv2.imshow("Cropped Prophesee", prophesee_cropped)
cv2.imshow("Cropped Overlay", overlay_cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save
cv2.imwrite("basler_warped_cropped.png", basler_warped_cropped)
cv2.imwrite("prophesee_cropped.png", prophesee_cropped)
cv2.imwrite("overlay_cropped.png", overlay_cropped)

print(f"Cropped region: x={x}, y={y}, w={w}, h={h}")

✅ Cropped region: x=114, y=0, w=1149, h=720


In [63]:
# =============
# Map homography to video frames (only mapping, no cropping)
# =============
#Paths
basler_folder = "Videos/basler_frames_12"
output_folder = "Warped_Basler_Frames_12"
os.makedirs(output_folder, exist_ok=True)

# Prophesee resolution
prophesee_width = 1280
prophesee_height = 720

# Loop through Basler images
for filename in os.listdir(basler_folder):
    if filename.endswith((".png", ".jpg", ".tiff")):
        basler_path = os.path.join(basler_folder, filename)
        basler_img = cv2.imread(basler_path)

        warped_img = cv2.warpPerspective(
            basler_img, H, (prophesee_width, prophesee_height)
        )

        cv2.imwrite(os.path.join(output_folder, filename), warped_img)

print("All Basler frames warped and saved!")

All Basler frames warped and saved!


In [81]:
# =============
# Process events for TimeLens (only do if prism has flipped prophesee video)
# =============
raw_file = "Videos/output_prophesee_12.raw"
output_events = []

# Create event iterator (read raw file)
mv_iterator = EventsIterator(raw_file)
width, height = mv_iterator.get_size() 

# Iterate through event slices
for events in mv_iterator:
    # Example flip horizontally
    events['x'] = width - 1 - events['x']

    # Example flip vertically
    events['y'] = height - 1 - events['y']

    output_events.append(events)

print("Done flipping all events!")

Done flipping all events!


In [65]:
width

720

In [66]:
height

1280

In [67]:
# =============
# Process basler chunk data into timestamps
# =============

# Clock frequency of Basler
tick_frequency = 1_000_000_000
basler_ts = []

# Paths
input_file = 'Videos/basler_chunk_timestamps_12.csv'
output_file = 'Videos/basler_chunk_timestamps_12_corrected.csv'

# Translate chunk data into timestamps (s)  
with open(input_file, 'r', newline='') as infile, \
     open(output_file, 'w', newline='') as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    header = next(reader)
    writer.writerow(header)
    # Make initial timestamp "0"
    basler_corrected_ts = [i - basler_ts[0] for i in basler_ts]

    # Write timestamps to csv for cross reference with event triggers
    for row in reader:
        frame_index = row[0]
        chunk_ticks = int(row[1])
        timestamp_sec = chunk_ticks / tick_frequency
        basler_ts.append(timestamp_sec)
        writer.writerow([frame_index, chunk_ticks, f"{timestamp_sec:.9f}"])
        
print(f"Corrected timestamps saved to {output_file}")

Corrected timestamps saved to Videos/basler_chunk_timestamps_12_corrected.csv


In [68]:
basler_corrected_ts = [i - basler_ts[0] for i in basler_ts]

In [69]:
len(basler_corrected_ts)/basler_corrected_ts[len(basler_corrected_ts)-1]

169.49442259087695

In [70]:
# =============
# Collect triggers from prophesee file and translate to timestamps
# =============

prophesee_path = "Videos/output_prophesee_12.raw"
mv_iterator = EventsIterator(prophesee_path, delta_t=1000)

all_triggers = []

for evs in mv_iterator:
    triggers = mv_iterator.reader.get_ext_trigger_events()
    if triggers.size > 0:  # Check size!
        all_triggers.extend(triggers)

print(f"Collected {len(all_triggers)} triggers.")

#Save to CSV
import csv
with open("prophesee_triggers_12.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Polarity", "Timestamp (us)", "Channel"])
    for trig in all_triggers:
        writer.writerow(trig)

print(f"Saved {len(all_triggers)} triggers.")

Collected 20784055 triggers.
Saved 20784055 triggers.


In [71]:
# =============
# Clean trigger timestamps
# =============

trigger_df = pd.read_csv("prophesee_triggers_12.csv")

# Make sure timestamps are in order
trigger_sort_df = trigger_df.sort_values(by="Timestamp (us)")

# Make sure trigger duplicates (wave bounces) are removed
trigger_clean_df = trigger_sort_df.drop_duplicates(subset=["Polarity", "Timestamp (us)"])

# Translate us to s for crossreferencing with Basler timestamps
trigger_clean_df["Timestamp (s)"] = trigger_clean_df["Timestamp (us)"] / 1_000_000

trigger_clean_df.to_csv("prophesee_triggers_12_clean.csv", index=False)

C:\Users\rowen\AppData\Local\Temp\ipykernel_19564\2640255954.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trigger_clean_df["Timestamp (s)"] = trigger_clean_df["Timestamp (us)"] / 1_000_000


In [72]:
# Isolate high triggers (when sensor is being opened)
trigger_rising_df = trigger_clean_df[trigger_clean_df["Polarity"] == 1].copy()
trigger_times = trigger_rising_df["Timestamp (us)"]

In [73]:
trigger_times_index = trigger_times.sort_values().reset_index(drop=True)

In [74]:
trigger_times_index

0         13319
1         19237
2         25156
3         31075
4         36988
         ...   
335     1995647
336     2001566
337     2007485
338     2013399
339    22780342
Name: Timestamp (us), Length: 340, dtype: int64

In [75]:
# =============
# Bin events into numpy arrays between each timestamp
# =============

raw_file = prophesee_path
output_folder = "event_slices_12"
os.makedirs(output_folder, exist_ok=True)

trigger_times = trigger_times_index

# Use flipped events if prism has flipped prophesee video
events_flipped = np.concatenate(output_events)
print(f"All flipped events: {events_flipped.shape}")

for i in range(len(trigger_times) - 1):
    t_start = trigger_times[i]
    t_end   = trigger_times[i+1]
    print(f"Bin {i}: {t_start} → {t_end}")

    mask = (events_flipped['t'] >= t_start) & (events_flipped['t'] < t_end)
    evs = events_flipped[mask]

    np.savez_compressed(
        os.path.join(output_folder, f"frame_{i:06d}.npz"),
        # X location
        x=evs['x'],
        # Y Location
        y=evs['y'],
        # Polarity
        p=evs['p'],
        # Time (us)
        t=evs['t']
    )

print(f"Done! Saved {len(trigger_times)-1} flipped event bins.")

All flipped events: (77498897,)
Bin 0: 13319 → 19237
Bin 1: 19237 → 25156
Bin 2: 25156 → 31075
Bin 3: 31075 → 36988
Bin 4: 36988 → 42907
Bin 5: 42907 → 48826
Bin 6: 48826 → 54745
Bin 7: 54745 → 60658
Bin 8: 60658 → 66577
Bin 9: 66577 → 72496
Bin 10: 72496 → 78410
Bin 11: 78410 → 84329
Bin 12: 84329 → 90248
Bin 13: 90248 → 96167
Bin 14: 96167 → 102080
Bin 15: 102080 → 107999
Bin 16: 107999 → 113918
Bin 17: 113918 → 119831
Bin 18: 119831 → 125750
Bin 19: 125750 → 131669
Bin 20: 131669 → 137588
Bin 21: 137588 → 143502
Bin 22: 143502 → 149421
Bin 23: 149421 → 155339
Bin 24: 155339 → 161253
Bin 25: 161253 → 167172
Bin 26: 167172 → 173091
Bin 27: 173091 → 179010
Bin 28: 179010 → 184923
Bin 29: 184923 → 190842
Bin 30: 190842 → 196761
Bin 31: 196761 → 202680
Bin 32: 202680 → 208593
Bin 33: 208593 → 214512
Bin 34: 214512 → 220431
Bin 35: 220431 → 226345
Bin 36: 226345 → 232264
Bin 37: 232264 → 238183
Bin 38: 238183 → 244102
Bin 39: 244102 → 250015
Bin 40: 250015 → 255934
Bin 41: 255934 → 261853

In [77]:
# =============
# Translate .tiff frames to .png
# =============

tiff_dir = "Warped_Basler_Frames_12"
png_dir = "png_frames_12"

os.makedirs(png_dir, exist_ok=True)

for tiff_file in glob.glob(os.path.join(tiff_dir, "*.tiff")):
    img = cv2.imread(tiff_file, cv2.IMREAD_UNCHANGED)
    filename = os.path.basename(tiff_file).replace(".tiff", ".png")
    cv2.imwrite(os.path.join(png_dir, filename), img)

In [78]:
# =============
# Save timestamps to textfile (important for TimeLens)
# =============

np.savetxt("event_slices_12/timestamp.txt", trigger_times, fmt='%d')
np.savetxt("png_frames_12/timestamp.txt", trigger_times, fmt='%d')

In [79]:
# =============
# Create text file for ffmpeg to reference while turning frames into video
# =============

# Load timestamps in microseconds
timestamps = np.loadtxt("png_frames_12/timestamp.txt")

# Calculate real frame durations (in seconds)
durations = (timestamps[1:] - timestamps[:-1]) / 1e6  # convert µs to sec

# Slowdown factor (e.g., 2 real seconds → 60 seconds = 30x)
slowdown_factor = 30
durations *= slowdown_factor

# Make sure to reference correct folder for frames
with open("frames_12.txt", "w") as f:
    for i in range(len(durations)):
        f.write(f"file 'Warped_Basler_Frames_12/frame_{i:05d}.tiff'\n")
        f.write(f"duration {durations[i]:.6f}\n")
    # Add last frame without duration (required by FFmpeg)
    f.write(f"file 'Warped_Basler_Frames_12/frame_{len(durations):05d}.tiff'\n")

In [80]:
# =============
# Console command for ffmpeg to turn frames into video 
# =============
!ffmpeg -f concat -safe 0 -i frames_12.txt -vsync vfr -pix_fmt yuv420p output_12.mp4

ffmpeg version 2025-06-16-git-e6fb8f373e-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.1.0 (Rev4, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjx